In [1]:
# some potentially interesting clusting info at https://nlp.stanford.edu/IR-book/html/htmledition/flat-clustering-1.html
# filter out "Photo/Illustration by"
# split on longer dash character
import re
import newspaper
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_article_type(url):
    ieee_article_regex = "^https://www\.spectrum\.ieee\.org/(.*)/.*?$"
    article_type_string = re.match(ieee_article_regex, url)
    if article_type_string is None:
        return []
    else:
        article_types = article_type_string.group(1).split("/")
        article_categories = [atype for atype in article_types if atype in ARTICLE_CATEGORIES]
        return article_categories

In [3]:
ARTICLE_CATEGORIES = ["aerospace","at-work","biomedical","computing","energy","consumer-electronics",
                      "geek-life","green-tech","tech-history","robotics","semiconductors","telecom","transportation"]

In [5]:
ieee_spectrum = newspaper.build("https://www.spectrum.ieee.org/", memoize_articles = False)

In [6]:
article_urls = [a.url for a in ieee_spectrum.articles]
article_types = list(map(get_article_type, article_urls))
articles = [a for a,t in zip(article_urls, article_types) if len(t) > 0]
categories = [at[0] for at in article_types if len(at) > 0]

In [7]:
target_articles = [newspaper.Article(url) for url in articles]

In [10]:
for t in target_articles:
    print(t.url)
    t.download()
    t.parse()

https://www.spectrum.ieee.org/riskfactor/computing/software/ftc-puts-uber-on-a-privacy-and-security-choke-chain
https://www.spectrum.ieee.org/view-from-the-valley/telecom/internet/where-is-hbo-silicon-valleys-real-pied-piper-look-in-troon-scotland
https://www.spectrum.ieee.org/tech-talk/telecom/security/report-nextlevel-cyberattacks-demand-data-clearinghouse
https://www.spectrum.ieee.org/view-from-the-valley/computing/software/hackathons-are-usually-dominated-by-men-this-one-was-just-for-girls
https://www.spectrum.ieee.org/video/robotics/robotics-software/robots-learn-to-speak-body-language
https://www.spectrum.ieee.org/video/computing/networks/what-happens-in-your-brain-when-you-learn-a-song
https://www.spectrum.ieee.org/video/semiconductors/nanotechnology/how-will-we-go-beyond-moores-law-experts-weigh-in
https://www.spectrum.ieee.org/video/computing/software/how-to-detect-a-gps-spoof-on-a-superyacht
https://www.spectrum.ieee.org/video/geek-life/profiles/tech-meets-art-capturing-life-

ArticleException: 

In [17]:
target_articles[100].text

'Photo: Rohanmkth/Wikipedia\n\nAt the huge Mobile World Congress (MWC) held this week in Barcelona, there seemed to be a question on the tip of everyone’s tongue regarding the prospects of 5G: What will be 5G’s killer app?\n\nIn the past, when a new mobile protocol was introduced, the aim seemed to be to address a perceived need. (Although the predictions have not always been accurate: 3G addressed an anticipated demand for video conferencing that never really materialized.) However, 5G seems somewhat different in that it will be addressing an overall trend of higher efficiency and more bandwidth rather than a specific application demand.\n\nWith no clearly targeted demand, it’s understandable why people are wondering what 5G’s killer app might be. In a meeting at the MWC with Julius Robson, the Chief Strategy Officer of the Small Cell Forum, he suggested the killer app for 5G may not come from the usual suspects: artificial intelligence or virtual reality.\n\n“I don’t think it’s so mu

In [18]:
article_data = {"URL":articles[0:100],"Category":categories[0:100],"ArticleText":[x.text for x in target_articles[0:100]]}
text_df = pd.DataFrame(article_data)
# need to add third column for all article text

In [21]:
from nltk.stem.snowball import SnowballStemmer
import nltk

In [ ]:
def tokenize_and_stem(text, stemmer):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens = [t for t in tokens if re.search]

In [27]:
stemmer = SnowballStemmer("english")

text = target_articles[0].text
tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
tokens = [t for t in tokens if re.search("[a-zA-Z]", t)]
stems = [stemmer.stem(t) for t in tokens]
len(set(stems))/len(stems)

0.3665377176015474